### RunnablePassthrough

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [1]:
from langchain_teddynote import logging

logging.langsmith("edu-CH13-LCEL-Advanced")

LangSmith 추적을 시작합니다.
[프로젝트명]
edu-CH13-LCEL-Advanced


### 데이터 전달하기

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    # 입력값 그대로 반환
    passed=RunnablePassthrough(),
    # num 과, ret(num * 3) 을 반환하는 Runnable
    extra=RunnablePassthrough.assign(ret=lambda x: x["num"] * 3),
    # modified(num + 1) 을 반환하는 Runnable
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'ret': 3}, 'modified': 2}

### 검색기 예제

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    [
        "테디는 랭체인 주식회사에서 근무를 하였습니다.",
        "셜리는 테디와 같은 회사에서 근무하였습니다.",
        "테디의 직업은 개발자입니다.",
        "셜리의 직업은 디자이너입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o")

def format_docs(docs):
  return "\n".join([doc.page_content for doc in docs])

retrieval_chain = (
  {"context": retriever, "question": RunnablePassthrough()}
  | prompt 
  | llm 
  | StrOutputParser()
)

In [4]:
retrieval_chain.invoke("테디의 직업은 무엇입니까?")

'테디의 직업은 개발자입니다.'

In [5]:
retrieval_chain.invoke("셜리의 직업은 무엇입니까?")

'셜리의 직업은 디자이너입니다.'